In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzles Introducción

Primero se consideran 5 preguntas que servirán como introduccion a la logica 
knight-knave para aquellos que no estén familiarizados con ella y como un
breve curso recordatorio para los que lo están.

#### 2. Is it possible for an inhabitant of the island to claim that he and his brother are both knaves?

This question has provoked a good deal of controversy! Some claim that anyone who says that he and his brother are both knaves is certainly claiming that he is a knave, which is not possible, as we have seen in the answer to Question 1. Therefore, they conclude, no inhabitant can claim that he and his brother are both knaves.

This argument is wrong! Suppose an inhabitant A is a knave and his brother B is a knight. Then it is false that he and his brother are both knaves, hence he, as a knave, is certainly capable of making that false statement. Therefore it is possible for an inhabitant to claim that he and his brother are both knaves, but only if he is a knave and his brother is a knight.

This illustrates a curious principle about the logic of lying and truth-telling: Normally, if a truthful person claims that both of two statements are true, then he will certainly claim that each of the statements is true separately. But with a constant liar, the matter is different. Consider the following two statements: (1) My brother is a knave; (2) I am a knave. A
knave could claim that (1) and (2) together are both true, provided his brother is actually a knight, but he cannot claim (1) and claim (2)  separately, since he cannot claim (2). Again, a knave could say: "I am a knave and two plus two is five, " but he cannot separately claim: (1) "I am a knave"; (2) "Two plus two is five."

In [8]:
/* 
Es posible para un habitante de la isla afirmar que él y su hermano son ambos knaves?¿

Si A es un knave y su hermano B es un Knight:
El predicado A y B son ambos knaves es FALSO, pero al ser A un knave (que siempre miente), podria afirmar que A y B 
son ambos knaves

Por tanto la signatura es:

x.dice[(x.knave and y.knave)] => (se cumple solo si) x.knave and y.knight

x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight)

*/

In [15]:
// POSIBLE SOLUCION - VERSION 3

def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight) = 
    ({xSay: x.Says[(x.Knave,y.Knave)] => 
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) =>
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_L1: x.Knight => x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = premises.P2[(x.Knave,y.Knave)](x)
                val x_L2: x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = x_L1(xKnight)
                val x_L3: (x.Knave,y.Knave) = x_L2(xSay)
                x_L3._1: x.Knave
                x_L3._2: y.Knave
                
                val x_L4: x.Knight => Nothing = x_L3._1
                (x_L3._1, x_L4(xKnight)) : (x.Knave,y.Knight)
            
            case Right(xKnave: x.Knave) =>
                
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_R1: x.Knave => x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = premises.P3[(x.Knave,y.Knave)](x)
                val x_R2: x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = x_R1(xKnave)
                //val x_R3: ¬[(x.Knave,y.Knave)] = x_R2(xSay)
                val x_R3: ((x.Knave,y.Knave)) => Nothing = x_R2(xSay)
                
                //Para usar la funcion x_R3 necesito una tupla con x.Knave que ya lo tengo y con y.Knave que no lo tengo
                //Por eso necesito usar la P1 con (y) y poder sacar el y.Knave
            
                (premises.P1(y) match{
                    // Siempre que se hace un pattern matching hay que poner ambas ramas
                    case Left(yKnight: y.Knight) => (xKnave, yKnight) : (x.Knave,y.Knight)
                    // En el caso de Left, ya tengo el y.Knight asi que he obtenido la tupla (x.Knave,y.Knight)
                    case Right(yKnave: y.Knave) => x_R3((xKnave, yKnave)) : (x.Knave,y.Knight)
                    // En el caso del Right, solo tengo el x.Knave pero NO el y.Knight, pero puedo obtener un Nothing 
                    // en lugar de la tupla (x.Knave,y.Knight), porque el tipo Nothing es equivalente al tipo (x.Knave,y.Knight)
                }): (x.Knave,y.Knight) // Esta definicion de datos se puede quitar
                
        }): (x.Knave,y.Knight)
    })

defined function puzzle2

In [16]:
// POSIBLE SOLUCION - VERSION 3 - SIMPLIFICADO

def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight) = 
    ({xSay => 
        (premises.P1(x) match{
            case Left(xKnight) =>
                val x_L1: x.Knight => x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = premises.P2[(x.Knave,y.Knave)](x)
                val x_L2: x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = x_L1(xKnight)
                val x_L3: (x.Knave,y.Knave) = x_L2(xSay)
                val x_L4: x.Knight => Nothing = x_L3._1 //Esto se puede obviar
                (x_L3._1, x_L4(xKnight)) //Obviando x_L4, podemos sacar x_L3._1(xKnight) : (x.Knave,y.Knight)
                (x_L3._1, x_L4(xKnight)) // x_L3._1(xKnight) : (x.Knave,y.Knight)
                
            
            case Right(xKnave) =>
                val x_R1: x.Knave => x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = premises.P3[(x.Knave,y.Knave)](x)
                val x_R2: x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = x_R1(xKnave)
                val x_R3: ((x.Knave,y.Knave)) => Nothing = x_R2(xSay)
                
                premises.P1(y) match{
                    case Left(yKnight) => (xKnave, yKnight) 
                    case Right(yKnave) => x_R3((xKnave, yKnave)) 
                }
                
        })
    })

defined function puzzle2

In [19]:
// Sugar version
/*
def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight) = 
    ({xSay => 
        (premises.P1(x) match{
            case Left(xKnight) =>
                val x_L1: x.Knight => x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = premises.P2[(x.Knave,y.Knave)](x)
                val x_L2: x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knave) = x_L1(xKnight)
                val x_L3: (x.Knave,y.Knave) = x_L2(xSay)
                val x_L4: x.Knight => Nothing = x_L3._1 //Esto se puede obviar
                (x_L3._1, x_L4(xKnight)) //Obviando x_L4, podemos sacar x_L3._1(xKnight) : (x.Knave,y.Knight)
            
            case Right(xKnave) =>
                val x_R1: x.Knave => x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = premises.P3[(x.Knave,y.Knave)](x)
                val x_R2: x.Says[(x.Knave,y.Knave)] => ¬[(x.Knave,y.Knave)] = x_R1(xKnave)
                val x_R3: ((x.Knave,y.Knave)) => Nothing = x_R2(xSay)
                
                premises.P1(y) match{
                    case Left(yKnight) => (xKnave, yKnight) 
                    case Right(yKnave) => x_R3((xKnave, yKnave)) 
                }
                
        })
    })

*/

def puzzle2(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): x.Says[(x.Knave,y.Knave)] => (x.Knave,y.Knight) = 
    xSaysXisKnaveYisKnave =>
    //({xSay =>
        premises.eitherKnightOrKnave(x).fold(
        //(premises.P1(x) match{    
            { xIsKnight => 
            //case Left(xKnight) =>
                //def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val aux1 = premises.noKnightLies(x)(xIsKnight)(xSaysXisKnaveYisKnave: x.Says[(x.Knave,y.Knave)]): (x.Knave,y.Knave)
                //aux._1: x.Knave y aux._2: y.Knave
                aux1._1(xIsKnight)
            },
            { xIsKnave => 
            //case Right(xKnave) =>
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val aux2 = premises.noKnaveTellsTruth(x)(xIsKnave)(xSaysXisKnaveYisKnave: x.Says[(x.Knave,y.Knave)]): Not[(x.Knave,y.Knave)]
                premises.eitherKnightOrKnave(y).fold(
                //premises.P1(y) match{
                    yIsKnight =>
                    //case Left(yKnight) => (xKnave, yKnight) 
                        (xIsKnave, yIsKnight),
                    yIsKnave =>
                    //case Right(yKnave) => x_R3((xKnave, yKnave)) 
                        aux2((xIsKnave, yIsKnave))
                )
            }
    )

cmd19.sc:8: type mismatch;
 found   : x.Says[(x.Knave, y.Knave)]
    (which expands to)  x.Says[(x.Knight => Nothing, y.Knight => Nothing)]
 required: x.Says[ammonite.$sess.cmd17.wrapper.cmd1.Not[?]]
    (which expands to)  x.Says[? => Nothing]
                val aux1 = premises.noKnightLies(x)(xIsKnight)(xSaysXisKnaveYisKnave: x.Says[(x.Knave,y.Knave)]): (x.Knight => Nothing, y.Knight => Nothing)
                                                                                    ^Compilation Failed

: 